In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0q0aZ4D44pNieXTr6_dmRBe-jNFlZz2FAjlg_mUOBraJ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'hacka-donotdelete-pr-ntdcsr9ga1im5m'
object_key = 'dataset_processed.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head(10)


,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,A,0,0,0,0,0,0,0,0,0,...,93,187,89,0,0,0,0,0,0,0
6,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from tensorflow.keras import layers, models

def create_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

input_shape = (28, 28, 1)
num_classes = 26           

model = create_model(input_shape, num_classes)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [3]:
labels = dataset['label']
pixels = dataset.drop('label', axis=1)
X = pixels.values.reshape(-1, 28, 28, 1)  # Reshape to (num_samples, 28, 28, 1)
X = X / 255.0 

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(labels)

In [5]:
y = y_encoded

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/20
435/435 [==============================] - 11s 22ms/step - loss: 2.1253 - accuracy: 0.3345 - val_loss: 1.7393 - val_accuracy: 0.4692
Epoch 2/20
435/435 [==============================] - 11s 24ms/step - loss: 1.6690 - accuracy: 0.4835 - val_loss: 1.3485 - val_accuracy: 0.5993
Epoch 3/20
435/435 [==============================] - 9s 21ms/step - loss: 1.3455 - accuracy: 0.5749 - val_loss: 1.1506 - val_accuracy: 0.6479
Epoch 4/20
435/435 [==============================] - 9s 21ms/step - loss: 1.1544 - accuracy: 0.6357 - val_loss: 0.9475 - val_accuracy: 0.7048
Epoch 5/20
435/435 [==============================] - 9s 21ms/step - loss: 1.0132 - accuracy: 0.6719 - val_loss: 0.9214 - val_accuracy: 0.7117
Epoch 6/20
435/435 [==============================] - 8s 19ms/step - loss: 0.9125 - accuracy: 0.7069 - val_loss: 0.7812 - val_accuracy: 0.7523
Epoch 7/20
435/435 [==============================] - 9s 21ms/step - loss: 0.8312 - accuracy: 0.7314 - val_loss: 0.7929 - val_accuracy: 0.75

In [9]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "0ZMGeZzTgJ_lY9izPitQh4uXzeIr02Z9RSgVIq7YhpVO"
}

client = APIClient(wml_credentials)

In [10]:
space_id = "3a881f81-9856-492a-9fd3-c790b94ac831"
client.set.default_space(space_id)

'SUCCESS'

In [11]:
model.save('model-2.hd5')

INFO:tensorflow:Assets written to: model-2.hd5/assets


INFO:tensorflow:Assets written to: model-2.hd5/assets


In [12]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Handwriting Recognition Model_2",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.14", 
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("tensorflow_rt24.1-py3.11")
}

model_details = client.repository.store_model(model='model-2.hd5', meta_props=metadata)
print("Model uploaded. Model ID:", model_details["metadata"]["id"])

Model uploaded. Model ID: d2cfbd86-7b89-4bbb-8a61-d2b071d7178a


In [13]:
deployment_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Handwriting Recognition Deployment_2",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment_details = client.deployments.create(
    artifact_uid=model_details["metadata"]["id"], 
    meta_props=deployment_metadata
)

print("Deployment created. Deployment ID:", deployment_details["metadata"]["id"])



#######################################################################################

Synchronous deployment creation for uid: 'd2cfbd86-7b89-4bbb-8a61-d2b071d7178a' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='97db027d-1bea-447a-89d4-253d1a011dff'
------------------------------------------------------------------------------------------------


Deployment created. Deployment ID: 97db027d-1bea-447a-89d4-253d1a011dff
